In [ ]:
import tensorflow as tf
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, SubprocVecEnv
import numpy as np
import matplotlib.pyplot as plt
import os


from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import HParam
from stable_baselines3.common.callbacks import EvalCallback, CallbackList
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
# Parametri dell'ambiente
hp_reset_noise_scale = 0.04759391555142866  # Scala del rumore quando l'ambiente viene resettato
hp_forward_reward_weight = 1.895790973082321  # Peso della ricompensa per il movimento in avanti
hp_ctrl_cost_weight = 1.5077149854491863  # Peso del costo di controllo del robot
hp_healthy_reward = 2.4052783506228486  # Ricompensa per mantenere lo stato "sano"
hp_contact_cost_weight = 6.97442897111752e-05  # Peso del costo per i contatti
hp_healthy_z_lower = 0.38253131966695203  # Altezza minima considerata "sana"
hp_healthy_z_upper = 1.1546046945435202  # Altezza massima considerata "sana"
hp_contact_force_min = -1.254958844476713  # Forza minima di contatto considerata
hp_contact_force_max = 0.9972723819502675  # Forza massima di contatto considerata
hp_healthy_z_range=(hp_healthy_z_lower, hp_healthy_z_upper),
hp_contact_force_range=(hp_contact_force_min, hp_contact_force_max)

# Parametri di ottimizzazione per TD3
hp_learning_rate = 0.0001500747188896999  # Learning rate per la rete di attore-critico
hp_learning_starts = 5000  # Numero di step prima di iniziare gli aggiornamenti della rete
hp_batch_size = 1024  # Dimensione del batch per SGD
hp_gamma = 0.943633865948463  # Fattore di sconto per il futuro reward
hp_tau = 0.00823476649680571  # Fattore di interpolazione per l'aggiornamento della rete target
hp_noise_std = 0.11889226516450727  # Deviazione standard del rumore di esplorazione
hp_noise_clip = 0.3182225104935072  # Clipping del rumore per la policy target smoothing
hp_policy_delay = 3  # Delay tra gli aggiornamenti della politica rispetto ai Q-network updates
hp_train_freq = 10  # Frequenza di aggiornamento della rete (ogni X step di interazione)
hp_gradient_steps = 8  # Numero di passi di aggiornamento eseguiti dopo ogni batch

# Parametri dell'azione (rumore per esplorazione)
hp_action_noise_mean = [0.0] * 8  # Media del rumore gaussiano per l'esplorazione
hp_action_noise_sigma = [0.3426192847562814] * 8  # Deviazione standard del rumore gaussiano

# Parametri globali
hp_num_envs = 6  # Numero di environment paralleli per il training
hp_total_timesteps = 3_000_000  # Numero totale di timesteps per l'addestramento
hp_episodes_evaluation = 200  # Numero di episodi usati per valutare la policy


In [ ]:
def make_env():
    """
    Crea e restituisce l'ambiente Ant-v5 dalla libreria Gymnasium.

    Questa funzione istanzia l'ambiente "Ant-v5", uno degli ambienti recenti e ben supportati
    in Gymnasium. I parametri usati sono:
    - reset_noise_scale (0.1): determina la scala del rumore quando l'ambiente viene resettato.
    - render_mode ('None'): indica che non verrà effettuato il rendering durante l'esecuzione.

    Ritorna:
        gym.Env: l'ambiente Ant-v5 inizializzato.
    """
    
    # Ant-v5 è l’ambiente più recente in Gymnasium.
    return gym.make("Ant-v5", 
                    reset_noise_scale=hp_reset_noise_scale, # scala del rumore quando l'ambiente viene resettato 
                    forward_reward_weight=hp_forward_reward_weight, # peso del reward per il movimento in avanti
                    ctrl_cost_weight=hp_ctrl_cost_weight, # peso del reward per il controllo
                    healthy_reward =hp_healthy_reward, # reward per la salute
                    contact_cost_weight=hp_contact_cost_weight,
                    healthy_z_range=hp_healthy_z_range,
                    contact_force_range=hp_contact_force_range,
                    render_mode='none')

In [ ]:
# 1. Creiamo un ambiente vettorializzato (Vectorized Environment)
# Utilizziamo DummyVecEnv per gestire più istanze dell'ambiente come se fossero una singola entità.
# Qui passiamo la funzione make_env (definita in un'altra cella) che crea l'ambiente "Ant-v5".
#env = DummyVecEnv([make_env])  


NUM_ENVS=hp_num_envs
env = SubprocVecEnv([make_env for _ in range(NUM_ENVS)])


# 2. Normalizziamo osservazioni (obs) e ricompense (reward)
# VecNormalize scala le osservazioni e le ricompense per stabilizzare l'allenamento.
# Parametri:
#   norm_obs=True   -> Abilita la normalizzazione delle osservazioni.
#   norm_reward=True -> Abilita la normalizzazione delle ricompense.
#   clip_obs=10.     -> Limita i valori normalizzati dell'osservazione a un range [-10, 10] per evitare estremi.
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

In [ ]:
model = TD3("MlpPolicy", env,
                learning_rate=hp_learning_rate,
                buffer_size=50000,
                learning_starts=hp_learning_starts,
                batch_size=hp_batch_size,
                gamma=hp_gamma,
                tau=hp_tau,
                action_noise=hp_action_noise_mean,
                policy_delay=hp_policy_delay,
                train_freq=hp_train_freq,
                gradient_steps=hp_gradient_steps,
                seed=42,
                verbose=0)


In [ ]:
eval_env = SubprocVecEnv([make_env for _ in range(NUM_ENVS)])

eval_env = VecNormalize(eval_env, norm_obs=True, norm_reward=True, clip_obs=10.)

In [ ]:
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path="./logs/best_model",
    log_path="./logs/",
    eval_freq=50000,
    deterministic=True,
    render=False
)

In [ ]:
# 4. Alleniamo il modello
# Il parametro total_timesteps indica il numero totale di iterazioni (o passi)
# che il modello eseguirà durante l'allenamento. Ogni timestep rappresenta un'interazione
# con l'ambiente in cui il modello esegue un'azione e riceve un feedback, che viene poi
# usato per aggiornare la sua politica interna.
total_timesteps = hp_total_timesteps  # Puoi aumentare questo valore per permettere al modello di acquisire più esperienza.
model.learn(total_timesteps=total_timesteps, callback=eval_callback)

In [ ]:
model.save("td3_Ant_model")
env.save("vecnormalize_Ant.pkl") 

In [ ]:
def evaluate_random_policy(env, episodes=hp_episodes_evaluation):
    """
    Valuta una policy casuale su un ambiente dato.

    Parametri:
    - env: L'ambiente di simulazione.
    - episodes: Numero di episodi da eseguire per la valutazione.

    Ritorna:
    - La ricompensa media e la deviazione standard delle ricompense ottenute.
    """
    total_rewards = []
    for _ in range(episodes):
        obs = env.reset()
        done = [False] * env.num_envs
        episode_rewards = np.zeros(env.num_envs)
        while not all(done):
            actions = [env.action_space.sample() for _ in range(env.num_envs)]
            obs, rewards, done, infos = env.step(actions)
            episode_rewards += rewards
        total_rewards.extend(episode_rewards)
    mean_reward_random = np.mean(total_rewards)
    # std_reward_random = np.std(total_rewards)
    return mean_reward_random